<a href="https://colab.research.google.com/github/aditya-007/Aggregating-distributed-storage-for-ancillary-services/blob/main/MILP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pulp as plp

In [ ]:
# Define the model
model = plp.LpProblem("PV-Battery_Ancillary_Services", plp.LpMaximize)

In [ ]:
# Parameters (simplified for the example)
T = 96  # Time intervals
N = 2000  # Number of users
P_min_up = 1  # MW (minimum power for up regulation)
P_min_down = 1  # MW (minimum power for down regulation)

In [ ]:
# Up and Down regulation bids (€/MWh)
Bid_UP = 120  # Up regulation bid price
Bid_DOWN = 40  # Down regulation bid price


In [ ]:
# Variables
P_imp = plp.LpVariable.dicts("P_imp", ((i, t) for i in range(N) for t in range(T)), lowBound=0, cat='Continuous')
P_exp = plp.LpVariable.dicts("P_exp", ((i, t) for i in range(N) for t in range(T)), lowBound=0, cat='Continuous')
P_char = plp.LpVariable.dicts("P_char", ((i, t) for i in range(N) for t in range(T)), lowBound=0, cat='Continuous')
P_dchar = plp.LpVariable.dicts("P_dchar", ((i, t) for i in range(N) for t in range(T)), lowBound=0, cat='Continuous')
SoC = plp.LpVariable.dicts("SoC", ((i, t) for i in range(N) for t in range(T)), lowBound=0, cat='Continuous')

In [ ]:
# Binary decision variables for regulation participation
model.z_up = Var(model.t, domain=Binary)  # Binary variable for up regulation
model.w_down = Var(model.t, domain=Binary)  # Binary variable for down regulation

In [1]:
# Objective: Maximize (Revenue from regulation) - (Cost of energy)
def obj_expression(model):
    # Up regulation remuneration
    revenue_up = sum(model.z_up[t] * Bid_UP * (sum(model.P_exp[i, t] for i in model.i)) for t in model.t)

    # Down regulation remuneration
    revenue_down = sum(model.w_down[t] * Bid_DOWN * (sum(model.P_exp[i, t] for i in model.i)) for t in model.t)

    # Cost of energy imported from the grid
    cost_energy = sum(sum(model.P_imp[i, t] * 180 for i in model.i) for t in model.t)  # 180 €/MWh import cost

    return revenue_up + revenue_down - cost_energy

model.obj = Objective(rule=obj_expression, sense=maximize)

NameError: name 'plp' is not defined

In [ ]:
# Constraints

# Power balance for each user at each time period
def power_balance_rule(model, i, t):
    return model.P_imp[i, t] - model.P_exp[i, t] + model.P_dchar[i, t] - model.P_char[i, t] == P_dem[i-1][t-1] - P_gen[i-1][t-1]

model.power_balance = Constraint(model.i, model.t, rule=power_balance_rule)

# SoC dynamics (battery state of charge over time)
def soc_balance_rule(model, i, t):
    if t == 1:
        return model.SoC[i, t] == battery_capacity / 2  # Initial SoC is 50% full
    else:
        return model.SoC[i, t] == model.SoC[i, t-1] + charge_efficiency * model.P_char[i, t] - model.P_dchar[i, t] / discharge_efficiency

model.soc_balance = Constraint(model.i, model.t, rule=soc_balance_rule)

# Battery charging and discharging limits
def charge_limit_rule(model, i, t):
    return model.P_char[i, t] <= 2  # Maximum charge rate (2 kW for example)

def discharge_limit_rule(model, i, t):
    return model.P_dchar[i, t] <= 2  # Maximum discharge rate (2 kW for example)

model.charge_limit = Constraint(model.i, model.t, rule=charge_limit_rule)
model.discharge_limit = Constraint(model.i, model.t, rule=discharge_limit_rule)

# Global up regulation constraint
def up_regulation_rule(model, t):
    return sum(model.P_imp[i, t] - model.P_exp[i, t] for i in model.i) <= P_base[t-1] - P_min_up + (1 - model.z_up[t]) * 1e6

model.up_regulation = Constraint(model.t, rule=up_regulation_rule)

# Global down regulation constraint
def down_regulation_rule(model, t):
    return sum(model.P_imp[i, t] - model.P_exp[i, t] for i in model.i) >= P_base[t-1] + P_min_down - (1 - model.w_down[t]) * 1e6

model.down_regulation = Constraint(model.t, rule=down_regulation_rule)

In [ ]:
# Solve the model
solver = SolverFactory('glpk')  # You can use any solver supported by Pyomo, e.g., 'cbc', 'glpk', 'cplex'
solver.solve(model)

In [ ]:
# Print results
print("Status:", model.obj())
for i in model.i:
    for t in model.t:
        print(f"User {i}, Time {t}, Import: {model.P_imp[i, t].value}, Export: {model.P_exp[i, t].value}, Charge: {model.P_char[i, t].value}, Discharge: {model.P_dchar[i, t].value}, SoC: {model.SoC[i, t].value}")

for t in model.t:
    print(f"Time {t}, Up Regulation: {model.z_up[t].value}, Down Regulation: {model.w_down[t].value}")